# Import Dependancies

In [0]:
#pip install --upgrade xgboost

In [0]:
import os
import sys
import csv

import pandas as pd
#pd.set_option('display.max_rows', None)
# pd.options.display.float_format = '{:, .2f}'.format
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns', 100)

import numpy as np
from numpy import save, load
from numpy import savez_compressed
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
import copy
import pickle

#from scipy.misc import comb, logsumexp
from sklearn.manifold import TSNE #a tool to visualize high dimensional data
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD # dimensionality reduction using truncated SVD (AKA LSA)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import accuracy_score


import xgboost as xgb
from sklearn.model_selection import GridSearchCV


import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.collocations import *
import string #python module
import re # python regex module
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize

import gensim
from gensim.models import Word2Vec

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

np.random.seed(0)

from sklearn.preprocessing import normalize
from functools import reduce

In [0]:
# load data | train set = mancon corpus features; test set = cord corpus features

path = "/content/drive/My Drive/CoronaWhy/Task-VT/sub_task_CONTRADICTION/Contradiction_drug_individual_claims_filtered(1)/all_engineered_contra_data.pkl"
with open (path, "rb") as outfile:
    data = pickle.load(outfile)

X_train = data[:17911]
y_train = pd.read_csv("/content/drive/My Drive/CoronaWhy/Task-VT/sub_task_CONTRADICTION/Contradiction_drug_individual_claims_filtered(1)/y_train.csv", header = None)
y_train = y_train.iloc[:,1]
#y_train = y_train.values.reshape(-1,1)

X_test = data[17911:]

# xgboost

In [3]:
boost_clf = xgb.XGBClassifier(verbosity = 2, max_depth = 6, tree_method= 'gpu_hist' ,predictor = 'gpu_predictor')
boost_clf.fit(X_train, y_train)

training_preds = boost_clf.predict(X_train)
test_preds = boost_clf.predict(X_test)

training_accuracy = accuracy_score(y_train, training_preds)
#test_accuracy = accuracy_score(y_test, test_preds)

print("training accuracy: {:.4}%" .format(training_accuracy * 100))
#print("test accuracy: {:.4}%" .format(test_accuracy * 100))

training accuracy: 99.84%


In [4]:
test_preds = pd.Series(test_preds).rename("prediction")
print(type(test_preds))
print(test_preds.shape)

<class 'pandas.core.series.Series'>
(28643,)


In [6]:
# join test preds with corpus dataset
corpus = pd.read_excel("/content/drive/My Drive/CoronaWhy/Task-VT/sub_task_CONTRADICTION/Contradiction_drug_individual_claims_filtered(1)/drug_individual_claims_similarity_annotated_v05.19.xlsx")
print(corpus.shape)
test_corpus_with_preds = pd.concat([corpus, test_preds], axis=1 )

(28643, 10)


In [8]:
test_corpus_with_preds.tail(1)

,Unnamed: 0,paper1_cord_uid,paper2_cord_uid,text1,text2,similarity_score,drugs1,drugs2,annotation,evaluated_for,prediction
28642,28642,zblitbo0,zph6r4il,"cc-by-nc-nd 4.0 international license it is made available under a is the author/funder, who has granted medrxiv a license to display the preprint in perpetuity.",attention to patients with covid-19 with non-classic symptoms should increase to protect health providers.,0.569444,"compro,factor ix,stimate,thrombin,ultiva,vasocon","lactate,ultiva",NaN,NaN,0


In [0]:
test_corpus_with_preds.to_excel("/content/drive/My Drive/CoronaWhy/Task-VT/sub_task_CONTRADICTION/Contradiction_drug_individual_claims_filtered(1)/test_corpus_with_preds.xlsx")